In [41]:
import os
import qdrant_client
from dotenv import load_dotenv
from langchain.vectorstores import Qdrant 
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.document_loaders.csv_loader import CSVLoader
from langchain.llms import OpenAI
from langchain.chains import RetrievalQA

In [13]:
# load enviorment variables

load_dotenv()

QDRANT_URL = os.getenv('QDRANT_URL')
QDRANT_API_KEY = os.getenv('QDRANT_API_KEY')
COLLECTION_NAME = os.getenv('QDRANT_COLLECTION_NAME')
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')

In [14]:
# create a qdrant client

client = qdrant_client.QdrantClient(
    url=QDRANT_URL,
    api_key=QDRANT_API_KEY
)


In [15]:
# create vector store

embeddings = OpenAIEmbeddings()

vector_store = Qdrant(
    client=client,
    collection_name=COLLECTION_NAME,
    embeddings=embeddings,
)

In [36]:
# load documents

file_path = os.getenv('FILE_PATH')

loader = CSVLoader(file_path=file_path, source_column=('rating'), csv_args={'delimiter': ','}, autodetect_encoding=True)
documents = loader.load()
documents = documents[:100]
# docs = get_chunks(documents)

print(f"Extracted {file_path} with {len(documents)} pages...")
print(documents[0].page_content)
print(documents[0].metadata)
print(documents[0].metadata['row'])

docs = [doc.page_content for doc in documents]

Extracted bigBasketProducts.csv with 100 pages...
index: 1
product: Garlic Oil - Vegetarian Capsule 500 mg
category: Beauty & Hygiene
sub_category: Hair Care
brand: Sri Sri Ayurveda
sale_price: 220
market_price: 220
type: Hair Oil & Serum
rating: 4.1
description: This Product contains Garlic Oil that is known to help proper digestion, maintain proper cholesterol levels, support cardiovascular and also build immunity.  For Beauty tips, tricks & more visit https://bigbasket.blog/
{'source': '4.1', 'row': 0}
0


In [38]:
# add documents to vector store

vector_store.add_texts(
    texts=docs,
)

['37794dd505ff45c3926c8b9414225cd4',
 'da805d8af6764d37a913d9aa1e4e0114',
 '0d0c979a891247cb8230c1a28d4da4c2',
 'b9163e4b6d0d47afbc7acbc6843b768a',
 '16a83dc7c4c64fdf853a2ca7b90a18d9',
 '6f35aed659504e30b620ae1439fbfd6f',
 '21966182c67a4b7a8122ea1ed2527ebc',
 '732163647e214d9c97b4efb0a8f925dc',
 '7401b867a54c41d59dbab40a2bcf93b3',
 'ae0cf95b485b4735a174540097219ea4',
 'b4b2ec479fbe44a4adb01b3911851b91',
 '5589e03c9d0a4b899352784d80053b2a',
 '2298d7b65ee9495a84c570d63500a72d',
 '7310a9b82a32478fb5a41ca2b377d2d4',
 '776eb8d951c846d5a53d22485f8e69e6',
 '7177a49368e342548e6264dd8c8457f6',
 '590e6b2477c84a63ae28e5a37854fcb6',
 'c9f5011ea5bb4b64b997647a7cb8e8d7',
 'b558a30de0894a9bba6f674166d879cf',
 '76a31f04c60442a9a14e6109a29f8526',
 '3106ee54f50a4986a9af3acfedad710f',
 '8552fa6c5ea5439bb97e2c38848e3544',
 '310568bb1de7472f86bda5c424f86808',
 '706bf031649b40fcb08224796a0f2655',
 '00fa72456edf4189bbe99fb5c61acdd0',
 'a57806f4d41e43cc92bfacbb10de7078',
 'e530e953fe284dabb91df5c238cfcaf1',
 

In [42]:
# plug vector store into retrieval chain

llm = OpenAI(openai_api_key=OPENAI_API_KEY)

qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vector_store.as_retriever(),
)

In [43]:
question = 'What is the best product for hair growth?'
print('>', question)
print(qa.run(question), end="\n\n")

> What is the best product for hair growth?
 It depends on your individual hair type and needs. Biotin & Collagen Volumizing Hair Shampoo + Biotin & Collagen Hair Conditioner and Argan-Liquid Gold Hair Spa both contain ingredients that help promote healthy hair growth, like Biotin, Hydrolyzed Collagen, and Argan Oil. It's best to do some research and find a product that best suits your individual hair type and needs.

